# начало работы

In [6]:
# === Block 0. Базовые импорты, конфиг, сиды ===

import os
import sys
import json
import math
import random
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch

# --- базовые пути (можно править под конкретное соревнование) ---
BASE_DIR = Path(".").resolve()
DATA_DIR = BASE_DIR / "data"          # сюда обычно кладём исходные данные
DOCS_DIR = DATA_DIR / "docs"          # сюда — документы для RAG (pdf/txt/html и т.д.)
OUTPUT_DIR = BASE_DIR / "outputs"     # сюда — артефакты, индекс, сабмиты
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- глобальный конфиг (можно потом расширять) ---
SEED = 42

# размер чанка и overlap будут важны для сплиттера,
# здесь просто задаём дефолт, позже ещё будет отдельный блок про чанкинг
CHUNK_SIZE = 512
CHUNK_OVERLAP = 64

# максимальная длина контекста, который будем скармливать модели
MAX_CONTEXT_CHARS = 4000

# заглушки для имён моделей (позже конкретизируем в отдельном блоке)
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
GENERATION_MODEL_NAME = "gpt2"  # пример; позже поменяем на нужную локальную модель

# --- функция для фиксации сида ---
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(SEED)

# --- информация о девайсе ---
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    DEVICE = torch.device("cpu")
    print("Using CPU")

print("BASE_DIR:", BASE_DIR)
print("DATA_DIR:", DATA_DIR)
print("DOCS_DIR:", DOCS_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)
print("SEED:", SEED)


Using GPU: Tesla P100-PCIE-16GB
BASE_DIR: /kaggle/working
DATA_DIR: /kaggle/working/data
DOCS_DIR: /kaggle/working/data/docs
OUTPUT_DIR: /kaggle/working/outputs
SEED: 42


# подгруз данных

In [7]:
# === Block 0.5. Копируем данные соревнования в нашу структуру ===

# создаём папки, если их ещё нет
DATA_DIR.mkdir(parents=True, exist_ok=True)
DOCS_DIR.mkdir(parents=True, exist_ok=True)

src_dir = Path("/kaggle/input/competition-psycho")

# копируем pdf в docs
src_book = src_dir / "book.pdf"
dst_book = DOCS_DIR / "book.pdf"

if src_book.exists():
    import shutil
    shutil.copy2(src_book, dst_book)
    print("Скопировал book.pdf →", dst_book)
else:
    print("НЕ НАШЁЛ файл:", src_book)

# копируем queries.json в корень data
src_queries = src_dir / "queries.json"
dst_queries = DATA_DIR / "queries.json"

if src_queries.exists():
    import shutil
    shutil.copy2(src_queries, dst_queries)
    print("Скопировал queries.json →", dst_queries)
else:
    print("НЕ НАШЁЛ файл:", src_queries)


Скопировал book.pdf → /kaggle/working/data/docs/book.pdf
Скопировал queries.json → /kaggle/working/data/queries.json


In [8]:
# === Block 1. Поиск документов для RAG в DOCS_DIR ===

# какие типы файлов считаем документами
SUPPORTED_EXTENSIONS = [".pdf", ".txt", ".md", ".html", ".htm"]

def list_documents(docs_dir: Path, exts=None) -> pd.DataFrame:
    # если не передали свой список расширений — используем дефолтный
    if exts is None:
        exts = SUPPORTED_EXTENSIONS

    docs = []

    # если папки с документами ещё нет — создаём пустую и сразу возвращаем пустой DataFrame
    if not docs_dir.exists():
        print("DOCS_DIR не существует, создаю пустую папку:", docs_dir)
        docs_dir.mkdir(parents=True, exist_ok=True)
        return pd.DataFrame(columns=["doc_id", "path", "ext"])

    # рекурсивно обходим DOCS_DIR и собираем все подходящие файлы
    for path in sorted(docs_dir.rglob("*")):
        if path.is_file():
            ext = path.suffix.lower()
            if ext in exts:
                docs.append(
                    {
                        "doc_id": len(docs),   # простой числовой id документа
                        "path": path,          # полный путь к файлу
                        "ext": ext,            # расширение файла
                    }
                )

    return pd.DataFrame(docs)


docs_df = list_documents(DOCS_DIR)

print("Найдено документов:", len(docs_df))
print(docs_df.head())


Найдено документов: 1
   doc_id                                path   ext
0       0  /kaggle/working/data/docs/book.pdf  .pdf


In [2]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 86.4 MB/s eta 0:00:00:00:0100:01


In [3]:
!uv pip install langchain-community

Using Python 3.11.13 environment at: /usr
Resolved 58 packages in 699ms                                        
Prepared 4 packages in 270ms                                             
Uninstalled 2 packages in 30ms
Installed 4 packages in 46ms0.0                             
 + langchain-classic==1.0.0
 + langchain-community==0.4.1
 - langchain-core==0.3.72
 + langchain-core==1.0.5
 - langchain-text-splitters==0.3.9
 + langchain-text-splitters==1.0.0


In [9]:
# === Block 2. Загрузка текста документов (pdf / txt / md / html) ===

from typing import List, Dict, Any

from langchain_community.document_loaders import PyPDFLoader  # для pdf через langchain


def load_pdf(path: Path) -> str:
    # читаем PDF постранично и склеиваем в один текст
    loader = PyPDFLoader(str(path))
    pages = loader.load()  # список объектов Document с .page_content
    texts = [p.page_content for p in pages]
    return "\n\n".join(texts)


def load_text_file(path: Path, encoding: str = "utf-8") -> str:
    # простой читатель для txt/md/html
    # (для html сейчас не чистим теги, просто читаем как есть — этого уже достаточно для baseline)
    with open(path, "r", encoding=encoding, errors="ignore") as f:
        return f.read()


def load_single_document(row: pd.Series) -> Dict[str, Any]:
    doc_id = int(row["doc_id"])
    path = Path(row["path"])
    ext = str(row["ext"]).lower()

    if ext == ".pdf":
        text = load_pdf(path)
    else:
        text = load_text_file(path)

    return {
        "doc_id": doc_id,
        "path": str(path),
        "ext": ext,
        "text": text,
        "n_chars": len(text),
    }


raw_docs: List[Dict[str, Any]] = []

if len(docs_df) == 0:
    print("В DOCS_DIR пока нет документов. Block 2 ничего не загрузил.")
else:
    print("Загружаю текст документов...")
    for _, row in tqdm(docs_df.iterrows(), total=len(docs_df)):
        doc_info = load_single_document(row)
        raw_docs.append(doc_info)

    print(f"Загружено документов: {len(raw_docs)}")

# переводим в DataFrame для удобства
raw_docs_df = pd.DataFrame(raw_docs)
print(raw_docs_df[["doc_id", "ext", "n_chars"]].head())


Загружаю текст документов...


100%|██████████| 1/1 [00:14<00:00, 14.52s/it]

Загружено документов: 1
   doc_id   ext  n_chars
0       0  .pdf  2256059


# чанкинг

In [10]:
# === Block 3. Чанкинг текста документов (семантический/структурный) ===

from typing import List, Dict, Any
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Минимальная доля пересечения между чанками (по символам)
MIN_OVERLAP_FRACTION = 0.25  # > 20% как ты просил

min_overlap = int(CHUNK_SIZE * MIN_OVERLAP_FRACTION)
EFFECTIVE_OVERLAP = max(CHUNK_OVERLAP, min_overlap)

print("CHUNK_SIZE:", CHUNK_SIZE)
print("CHUNK_OVERLAP (запрошенный):", CHUNK_OVERLAP)
print("EFFECTIVE_OVERLAP (используемый, >= 20%):", EFFECTIVE_OVERLAP)

# Сепараторы подобраны так, чтобы сперва резать по "крупной структуре":
#   - пустые строки / параграфы
#   - одиночные переводы строки
#   - предложения ". "
#   - пробелы
#   - в крайнем случае — посимвольно
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=EFFECTIVE_OVERLAP,
    separators=["\n\n", "\n", ". ", " ", ""],
    length_function=len,
)

def split_doc_to_chunks(doc_row: pd.Series) -> List[Dict[str, Any]]:
    doc_id = int(doc_row["doc_id"])
    text = str(doc_row["text"])
    path = str(doc_row["path"])
    ext = str(doc_row["ext"])

    if not text.strip():
        return []

    # создаём список "документов" langchain с общими метаданными
    lc_docs = text_splitter.create_documents(
        texts=[text],
        metadatas=[{
            "doc_id": doc_id,
            "path": path,
            "ext": ext,
        }],
    )

    chunks: List[Dict[str, Any]] = []
    for local_idx, d in enumerate(lc_docs):
        chunk_text = d.page_content
        meta = d.metadata or {}
        chunks.append(
            {
                "chunk_id": None,  # заполним позже глобальными id
                "doc_id": meta.get("doc_id", doc_id),
                "chunk_idx_in_doc": local_idx,
                "path": meta.get("path", path),
                "ext": meta.get("ext", ext),
                "text": chunk_text,
                "n_chars": len(chunk_text),
            }
        )
    return chunks


all_chunks: List[Dict[str, Any]] = []

if raw_docs_df is None or len(raw_docs_df) == 0:
    print("raw_docs_df пуст — сначала нужно загрузить документы (Block 2).")
else:
    print("Делаю чанкинг документов...")
    for _, row in tqdm(raw_docs_df.iterrows(), total=len(raw_docs_df)):
        doc_chunks = split_doc_to_chunks(row)
        all_chunks.extend(doc_chunks)

    # присваиваем глобальные chunk_id
    for idx, ch in enumerate(all_chunks):
        ch["chunk_id"] = idx

    print(f"Всего чанков: {len(all_chunks)}")

chunks_df = pd.DataFrame(all_chunks)

print("Размер chunks_df:", chunks_df.shape)
print(chunks_df[["chunk_id", "doc_id", "chunk_idx_in_doc", "n_chars"]].head())


CHUNK_SIZE: 512
CHUNK_OVERLAP (запрошенный): 64
EFFECTIVE_OVERLAP (используемый, >= 20%): 128
Делаю чанкинг документов...


100%|██████████| 1/1 [00:00<00:00, 11.01it/s]

Всего чанков: 6406
Размер chunks_df: (6406, 7)
   chunk_id  doc_id  chunk_idx_in_doc  n_chars
0         0       0                 0      175
1         1       0                 1      484
2         2       0                 2      422
3         3       0                 3      439
4         4       0                 4      501


# эмбединги чанков

In [17]:
!uv pip install faiss-cpu

'''по факту нужен gpu но на кагле он сдохнет с зависимостями и не может установиться'''


Using Python 3.11.13 environment at: /usr
Audited 1 package in 107ms


In [18]:
# === Block 4A. Инициализация эмбеддинг-модели и helper-функции ===

from sentence_transformers import SentenceTransformer
import faiss  # сам индекс будем делать в следующем блоке

# при желании можно поменять модель на другую из sentence-transformers
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

print("Загружаю embedding-модель:", EMBEDDING_MODEL_NAME)
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=str(DEVICE))

# Проверим размерность эмбеддингов на одном примере
test_emb = embed_model.encode(["test"], convert_to_numpy=True, show_progress_bar=False)
EMBED_DIM = int(test_emb.shape[1])
print("EMBED_DIM:", EMBED_DIM)


def embed_texts(texts, batch_size: int = 64):
    # texts — список строк
    # возвращает numpy-массив размера [len(texts), EMBED_DIM]
    return embed_model.encode(
        texts,
        batch_size=batch_size,
        convert_to_numpy=True,
        show_progress_bar=True,
        normalize_embeddings=True,  # сразу L2-нормализация, удобно для cosine/IP
    )

print("embed_texts() готова к использованию")


Загружаю embedding-модель: sentence-transformers/all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

EMBED_DIM: 384
embed_texts() готова к использованию


In [19]:
# === Block 4B. Эмбеддинги чанков + построение FAISS-индекса ===

if "chunks_df" not in globals() or chunks_df is None or len(chunks_df) == 0:
    print("chunks_df пуст — нужно выполнить блоки 2 и 3 (загрузка и чанкинг документов).")
else:
    chunk_texts = chunks_df["text"].astype(str).tolist()
    print("Чанков для эмбеддинга:", len(chunk_texts))

    # считаем эмбеддинги
    chunk_embs = embed_texts(chunk_texts, batch_size=64)  # можно подкрутить размер батча
    print("Готово, форма эмбеддингов:", chunk_embs.shape)

    # на всякий случай убеждаемся, что размерность совпадает
    assert chunk_embs.shape[1] == EMBED_DIM

    # создаём FAISS-индекс под cosine similarity:
    # мы уже нормализовали в embed_texts(normalize_embeddings=True),
    # поэтому можно использовать скалярное произведение (IP).
    index = faiss.IndexFlatIP(EMBED_DIM)

    # если доступен GPU-FAISS, можно перенести индекс на GPU
    faiss_res = None
    if torch.cuda.is_available():
        try:
            faiss_res = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(faiss_res, 0, index)
            print("FAISS-индекс перенесён на GPU")
        except Exception as e:
            print("Не удалось перенести FAISS-индекс на GPU, остаёмся на CPU:", e)

    # добавляем в индекс все вектора
    index.add(chunk_embs)
    print("Векторов в FAISS-индексе:", index.ntotal)

    # сохраняем эмбеддинги и метаданные на диск
    emb_path = OUTPUT_DIR / "chunk_embeddings.npy"
    meta_path = OUTPUT_DIR / "chunks_metadata.parquet"
    faiss_path = OUTPUT_DIR / "faiss_index.bin"

    np.save(emb_path, chunk_embs)
    chunks_df.to_parquet(meta_path, index=False)

    # для сохранения индекса на диск он должен быть на CPU
    index_cpu = index
    if faiss_res is not None:
        try:
            index_cpu = faiss.index_gpu_to_cpu(index)
        except Exception as e:
            print("Не удалось скопировать индекс с GPU на CPU, сохраняю как есть (может не сработать):", e)

    faiss.write_index(index_cpu, str(faiss_path))

    print("Эмбеддинги сохранены в:", emb_path)
    print("Метаданные чанков сохранены в:", meta_path)
    print("FAISS-индекс сохранён в:", faiss_path)


Чанков для эмбеддинга: 6406


Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Готово, форма эмбеддингов: (6406, 384)
Не удалось перенести FAISS-индекс на GPU, остаёмся на CPU: module 'faiss' has no attribute 'StandardGpuResources'
Векторов в FAISS-индексе: 6406
Эмбеддинги сохранены в: /kaggle/working/outputs/chunk_embeddings.npy
Метаданные чанков сохранены в: /kaggle/working/outputs/chunks_metadata.parquet
FAISS-индекс сохранён в: /kaggle/working/outputs/faiss_index.bin


# FAISS + BM25

In [21]:
!uv pip install rank_bm25

Using Python 3.11.13 environment at: /usr
⠙ Resolving dependencies...                                                     

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Resolved 14 packages in 85ms                                         
Prepared 1 package in 16ms                                               
Installed 1 package in 1ms                                  
 + rank-bm25==0.2.2


In [22]:
# === Block 5A. Построение BM25-индекса по чанкам ===

import re
from rank_bm25 import BM25Okapi

def simple_tokenize(text: str):
    # простой токенайзер:
    #   - в нижний регистр
    #   - разбиваем по "не-буквенно-цифровым" символам
    #   - выкидываем пустые токены
    text = text.lower()
    tokens = re.split(r"[^a-zA-Z0-9а-яА-ЯёЁ]+", text)
    tokens = [t for t in tokens if t]
    return tokens

if "chunks_df" not in globals() or chunks_df is None or len(chunks_df) == 0:
    print("chunks_df пуст — нужно выполнить блоки с загрузкой и чанкингом (2, 3A, 3B).")
else:
    print("Готовлю корпус для BM25 по чанкам...")
    
    # список текстов чанков
    bm25_texts = chunks_df["text"].astype(str).tolist()
    # токенизированный корпус
    bm25_corpus_tokens = [simple_tokenize(t) for t in tqdm(bm25_texts)]
    
    # сохраняем соответствие: позиция в BM25 ↔ chunk_id
    bm25_chunk_ids = chunks_df["chunk_id"].tolist()
    
    print("Строю BM25Okapi...")
    bm25 = BM25Okapi(bm25_corpus_tokens)
    
    print("BM25-индекс готов")
    print("Количество документов в BM25:", len(bm25_corpus_tokens))


Готовлю корпус для BM25 по чанкам...


100%|██████████| 6406/6406 [00:00<00:00, 30581.91it/s]


Строю BM25Okapi...
BM25-индекс готов
Количество документов в BM25: 6406


In [23]:
# === Block 5B. Гибридный поиск по чанкам: FAISS + BM25 ===

from typing import List, Tuple

def search_faiss_chunks(query: str, top_k: int = 20) -> pd.DataFrame:
    # эмбеддим запрос
    q_emb = embed_texts([query], batch_size=1)  # (1, EMBED_DIM)
    
    # ищем векторно
    scores, indices = index.search(q_emb, top_k)  # scores: (1, top_k), indices: (1, top_k)
    scores = scores[0]
    indices = indices[0]
    
    # индексы здесь — позиции в массиве chunk_embs, которые соответствуют строкам chunks_df по порядку
    # мы использовали chunks_df["text"].tolist() в Block 4B, так что соответствие такое:
    #   pos i  <->  chunks_df.iloc[i]
    
    rows = []
    for pos, score in zip(indices, scores):
        if pos < 0:
            continue
        row = chunks_df.iloc[int(pos)]
        rows.append(
            {
                "chunk_id": int(row["chunk_id"]),
                "doc_id": int(row["doc_id"]),
                "score_faiss": float(score),
            }
        )
    return pd.DataFrame(rows)


def search_bm25_chunks(query: str, top_k: int = 20) -> pd.DataFrame:
    if "bm25" not in globals():
        raise RuntimeError("BM25-индекс не найден. Сначала выполните Block 5A.")
    
    # токенизируем запрос
    q_tokens = simple_tokenize(query)
    if not q_tokens:
        return pd.DataFrame(columns=["chunk_id", "doc_id", "score_bm25"])
    
    # получаем bm25-оценки для каждого документа (по позиции)
    scores = bm25.get_scores(q_tokens)  # numpy-массив длины len(bm25_corpus_tokens)
    
    # берём top_k позиций
    top_k = min(top_k, len(scores))
    top_indices = np.argsort(scores)[::-1][:top_k]
    
    rows = []
    for pos in top_indices:
        score = float(scores[pos])
        chunk_id = int(bm25_chunk_ids[pos])
        # doc_id берём из chunks_df
        row = chunks_df.loc[chunks_df["chunk_id"] == chunk_id].iloc[0]
        rows.append(
            {
                "chunk_id": chunk_id,
                "doc_id": int(row["doc_id"]),
                "score_bm25": score,
            }
        )
    return pd.DataFrame(rows)


def hybrid_search_chunks(
    query: str,
    top_k_faiss: int = 20,
    top_k_bm25: int = 20,
    alpha_faiss: float = 0.5,
) -> pd.DataFrame:
    # alpha_faiss — вес векторного поиска,
    # (1 - alpha_faiss) — вес BM25
    
    df_faiss = search_faiss_chunks(query, top_k=top_k_faiss)
    df_bm25 = search_bm25_chunks(query, top_k=top_k_bm25)
    
    # нормализуем оценки, чтобы они были в сопоставимом масштабе (0..1)
    if len(df_faiss) > 0:
        max_f = max(abs(df_faiss["score_faiss"].max()), 1e-9)
        df_faiss["score_faiss_norm"] = df_faiss["score_faiss"] / max_f
    else:
        df_faiss["score_faiss_norm"] = []
    
    if len(df_bm25) > 0:
        max_b = max(abs(df_bm25["score_bm25"].max()), 1e-9)
        df_bm25["score_bm25_norm"] = df_bm25["score_bm25"] / max_b
    else:
        df_bm25["score_bm25_norm"] = []
    
    # объединяем по chunk_id
    hybrid = pd.merge(
        df_faiss[["chunk_id", "doc_id", "score_faiss_norm"]],
        df_bm25[["chunk_id", "score_bm25_norm"]],
        on="chunk_id",
        how="outer",
        suffixes=("_faiss", "_bm25"),
    )
    
    # doc_id может быть NaN, если чанка не было в faiss-результатах, но был в bm25
    if "doc_id" not in hybrid.columns:
        hybrid["doc_id"] = np.nan
    else:
        hybrid["doc_id"] = hybrid["doc_id"].fillna(-1).astype(int)
    
    hybrid["score_faiss_norm"] = hybrid["score_faiss_norm"].fillna(0.0)
    hybrid["score_bm25_norm"] = hybrid["score_bm25_norm"].fillna(0.0)
    
    # итоговый скор = alpha * faiss + (1 - alpha) * bm25
    hybrid["score_hybrid"] = (
        alpha_faiss * hybrid["score_faiss_norm"]
        + (1.0 - alpha_faiss) * hybrid["score_bm25_norm"]
    )
    
    # сортируем по итоговому скору
    hybrid = hybrid.sort_values("score_hybrid", ascending=False)
    
    # забираем top-k кандидатов
    hybrid_top = hybrid.head(max(top_k_faiss, top_k_bm25)).reset_index(drop=True)
    
    return hybrid_top


print("Функции search_faiss_chunks, search_bm25_chunks и hybrid_search_chunks готовы.")


Функции search_faiss_chunks, search_bm25_chunks и hybrid_search_chunks готовы.


# груз ллм

In [24]:
'''# === Block 6A. Загрузка LLM с Hugging Face (Qwen2.5-1.5B-Instruct по умолчанию) ===

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Hugging Face ID модели по умолчанию
DEFAULT_HF_MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

# можно переопределить через переменную окружения, если захочешь
HF_MODEL_ID = os.environ.get("RAG_LLM_ID", DEFAULT_HF_MODEL_ID)

# папка для локального кеша моделей (внутри Permanent)
MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# отдельная подпапка под конкретную модель
LOCAL_MODEL_DIR = MODELS_DIR / HF_MODEL_ID.replace("/", "_")
LOCAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("HF_MODEL_ID:", HF_MODEL_ID)
print("LOCAL_MODEL_DIR:", LOCAL_MODEL_DIR)

# при первом вызове from_pretrained модель скачается в LOCAL_MODEL_DIR,
# дальше будет использовать локальный кеш, даже если интернет отвалится
print("Загружаю tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    HF_MODEL_ID,
    cache_dir=str(LOCAL_MODEL_DIR),
    trust_remote_code=True
)

print("Загружаю модель (это может занять время при первом запуске)...")
model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL_ID,
    cache_dir=str(LOCAL_MODEL_DIR),
    trust_remote_code=True,
    torch_dtype=torch.float16,   # на A100 это ок
    device_map="auto"           # автоматически раскидывает слои по доступным устройствам
)

gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)

print("LLM загружена и готова к генерации.")'''


HF_MODEL_ID: Qwen/Qwen2.5-1.5B-Instruct
LOCAL_MODEL_DIR: /kaggle/working/models/Qwen_Qwen2.5-1.5B-Instruct
Загружаю tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Загружаю модель (это может занять время при первом запуске)...


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object.

In [80]:
# === Block 6A. Загрузка LLM с Hugging Face (фиксанутая версия для Kaggle) ===

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

DEFAULT_HF_MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
HF_MODEL_ID = os.environ.get("RAG_LLM_ID", DEFAULT_HF_MODEL_ID)

MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

LOCAL_MODEL_DIR = MODELS_DIR / HF_MODEL_ID.replace("/", "_")
LOCAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)

print("HF_MODEL_ID:", HF_MODEL_ID)
print("LOCAL_MODEL_DIR:", LOCAL_MODEL_DIR)

print("Загружаю tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    HF_MODEL_ID,
    cache_dir=str(LOCAL_MODEL_DIR),
    trust_remote_code=True,
)

print("Загружаю модель (это может занять время при первом запуске)...")
model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL_ID,
    cache_dir=str(LOCAL_MODEL_DIR),
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",   # accelerate сам раскинет по девайсу
)

# ВАЖНО: НЕ передаём device=..., иначе будет тот самый ValueError
gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("LLM загружена и готова к генерации.")


HF_MODEL_ID: Qwen/Qwen2.5-1.5B-Instruct
LOCAL_MODEL_DIR: /kaggle/working/models/Qwen_Qwen2.5-1.5B-Instruct
Загружаю tokenizer...
Загружаю модель (это может занять время при первом запуске)...


Device set to use cuda:0


LLM загружена и готова к генерации.


In [ ]:
'''HF_MODEL_ID = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_ID, cache_dir=str(LOCAL_MODEL_DIR), trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL_ID,
    cache_dir=str(LOCAL_MODEL_DIR),
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto",
)'''

In [ ]:
'''HF_MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"'''

In [99]:
''' HF_MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct" '''

'''1. LLaMA-3.1-8B-Instruct — лучший вариант

HF ID: meta-llama/Meta-Llama-3.1-8B-Instruct
VRAM: 12–16 GB (fp16), 8–10 GB (fp8/GPTQ)
Почему выбрать:

стабильная, ровная, очень сильная на английском;

работает лучше Qwen-2.5-7B в reasoning и factual QA;

идеально подходит для RAG, умеет следовать структуре.'''


In [ ]:
'''HF_MODEL_ID = "Qwen/Qwen2.5-14B-Instruct"
'''

'''2. Qwen2.5-14B-Instruct — сильнее 7B почти во всём

HF ID: Qwen/Qwen2.5-14B-Instruct
VRAM: 28–32 GB (fp16), ~18 GB (AWQ / GPTQ)
Плюсы:

мощная reasoning-модель;

отличное качество ответов + хорошая устойчивость к галлюцинациям;

один из лучших вариантов под англоязычный RAG.'''

In [ ]:
'''HF_MODEL_ID = "mistralai/Mistral-Nemo-12B-Instruct"
'''

'''3. Mistral-NeMo-12B-Instruct — очень хороший баланс

HF ID: mistralai/Mistral-Nemo-12B-Instruct
VRAM: 14–18 GB (fp16)
Плюсы:

сильный reasoning;

хорошая работа с длинными документами;

быстрый инференс.'''

In [ ]:
'''HF_MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B-GPTQ"
'''

'''4. DeepSeek-R1-Distill-LLaMA-70B (GPTQ) — почти GPT-4 уровень

HF ID:
deepseek-ai/DeepSeek-R1-Distill-Llama-70B-GPTQ
или
unsloth/DeepSeek-R1-Distill-Llama-70B-bnb-4bit

VRAM: 18–28 GB (4-bit)
Плюсы:

супер-reasoning, очень аккуратные аргументы;

минимальные галлюцинации;

идеальна для RAG c длинными документами.'''

In [100]:
'''Параметры загрузки (копировать в Block 6A)'''

'''from transformers import AutoTokenizer, AutoModelForCausalLM

HF_MODEL_ID = "meta-llama/Meta-Llama-3.1-8B-Instruct"   # ← меняешь ID здесь

tokenizer = AutoTokenizer.from_pretrained(
    HF_MODEL_ID,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
'''

'''Для GPTQ/AWQ:'''
'''model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL_ID,
    device_map="auto",
    torch_dtype="auto",
    low_cpu_mem_usage=True
)
'''

'''Что делать, если не знаешь, какую модель выбрать:

Сначала пробуй LLaMA-3.1-8B-Instruct
— лучший «универсал» под RAG.

Если остаётся VRAM → переходи на Qwen2.5-14B (GPTQ).

Если есть реальный запас VRAM → DeepSeek-R1-70B (GPTQ).

Если интернет медленный: бери Qwen2.5-7B-Instruct (самый лёгкий скачиваемый сильный вариант).'''

'Для GPTQ/AWQ:'

функция генерации

In [81]:
# === Block 6B. Функция generate_answer() поверх gen_pipe ===

from typing import Optional

def generate_answer(
    prompt: str,
    max_new_tokens: int = 256,
    temperature: float = 0.2,
    top_p: float = 0.9,
    do_sample: bool = True,
    stop_token: Optional[str] = None,
):
    """
    Обёртка над gen_pipe.
    На вход: готовый prompt (обычно system+context+question).
    На выход: одна строка с ответом модели.
    """
    # gen_pipe возвращает список объектов вида:
    # [{"generated_text": "..."}]
    outputs = gen_pipe(
        prompt,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=top_p,
        do_sample=do_sample,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    full_text = outputs[0]["generated_text"]

    # Многие модели возвращают "prompt + continuation".
    # Простейший способ отделить — отрезать префикс prompt, если он совпадает.
    if full_text.startswith(prompt):
        answer = full_text[len(prompt):]
    else:
        answer = full_text

    # Если задан stop_token — обрезаем по нему (например, "\n\nUser:" или т.п.)
    if stop_token is not None and stop_token in answer:
        answer = answer.split(stop_token, 1)[0]

    return answer.strip()

print("Функция generate_answer() определена.")


Функция generate_answer() определена.


# вытаскивание релевантных чанков и сбор промта

In [82]:
# === Block 7A. Поиск релевантных чанков и сбор контекста ===

def retrieve_relevant_chunks(
    query: str,
    top_k_faiss: int = 20,
    top_k_bm25: int = 20,
    top_k_final: int = 8,
    alpha_faiss: float = 0.6,
    max_context_chars: int = MAX_CONTEXT_CHARS,
):
    """
    Возвращает список словарей с полями:
    - chunk_id
    - doc_id
    - text
    - score_hybrid
    + один большой context_text, обрезанный по max_context_chars.
    """
    # Гибридный поиск по чанкам (faiss + bm25)
    hybrid = hybrid_search_chunks(
        query=query,
        top_k_faiss=top_k_faiss,
        top_k_bm25=top_k_bm25,
        alpha_faiss=alpha_faiss,
    )

    # Берём только top_k_final чанков
    hybrid = hybrid.head(top_k_final).reset_index(drop=True)

    # join с chunks_df, чтобы достать текст
    merged = hybrid.merge(
        chunks_df[["chunk_id", "doc_id", "text"]],
        on=["chunk_id", "doc_id"],
        how="left",
    )

    # Сортируем по итоговому скору ещё раз (на всякий случай)
    merged = merged.sort_values("score_hybrid", ascending=False).reset_index(drop=True)

    # Собираем контекст с ограничением по длине
    context_pieces = []
    total_len = 0

    selected_chunks = []

    for _, row in merged.iterrows():
        chunk_text = str(row["text"])
        chunk_len = len(chunk_text)

        if total_len + chunk_len > max_context_chars:
            # если совсем ничего ещё не добавили — всё равно добавим первый
            if not context_pieces:
                context_pieces.append(chunk_text[:max_context_chars])
                selected_chunks.append(
                    {
                        "chunk_id": int(row["chunk_id"]),
                        "doc_id": int(row["doc_id"]),
                        "text": chunk_text[:max_context_chars],
                        "score_hybrid": float(row["score_hybrid"]),
                    }
                )
            break

        context_pieces.append(chunk_text)
        total_len += chunk_len

        selected_chunks.append(
            {
                "chunk_id": int(row["chunk_id"]),
                "doc_id": int(row["doc_id"]),
                "text": chunk_text,
                "score_hybrid": float(row["score_hybrid"]),
            }
        )

    context_text = "\n\n".join(context_pieces)

    return {
        "chunks": selected_chunks,
        "context_text": context_text,
    }

print("Функция retrieve_relevant_chunks() определена.")


Функция retrieve_relevant_chunks() определена.


In [ ]:
'''instruction = (
    "You are an expert in document analysis and RAG.\n"
    "Your task is to answer questions strictly based on the provided context.\n"
    "Answer in English, clearly and in a structured way.\n"
    "Do not invent facts that are not present in the context.\n"
    "If the information is insufficient, say so explicitly.\n\n"
    "Answer format:\n"
    "1) SHORT ANSWER — 1–3 sentences, the main idea.\n"
    "2) REASONING — detailed explanation referring to key parts of the context (paraphrased).\n"
    "3) MISSING INFORMATION — what exactly cannot be answered from this context.\n"
)'''

'''"Answer strictly in the following structure (no extra sections):\n"
"SHORT ANSWER:\n"
"- ...\n\n"
"REASONING:\n"
"- ...\n\n"
"MISSING INFORMATION:\n"
"- ...\n\n"
"=== MODEL ANSWER ===\n"
'''

In [ ]:
'''в answer_question и rag_answer_with_context_and_refs вернуть:'''
'''return answer.strip() вместо normalize_rag_answer(...).'''

In [83]:
# === Block 7B (новая версия). Сбор промпта и answer_question() с чёткой структурой ===

'''def build_rag_prompt(
    query: str,
    context_text: str,
    instruction: str = None,
):
    """
    Собирает финальный текстовый prompt для LLM.
    Ролевая инструкция + жёсткая структура ответа.
    """
    if instruction is None:
        instruction = (
            "Ты опытный эксперт по анализу документов и задачам RAG.\n"
            ".\n"
            "Твоя задача — отвечать на вопросы строго на основе предоставленного контекста.\n"
            "Отвечай на русском языке, чётко и структурированно.\n"
            "Нельзя придумывать факты, которых нет в контексте.\n"
            "Если информации недостаточно, честно сообщи об этом.\n\n"
            "Формат ответа:\n"
            "1) КРАТКИЙ ОТВЕТ — 1–3 предложения, самая суть.\n"
            "2) ОБОСНОВАНИЕ — подробное пояснение со ссылкой на ключевые фрагменты контекста (пересказ, а не дословная цитата).\n"
            "3) НЕДОСТАЮЩИЕ ДАННЫЕ — что именно невозможно ответить по этому контексту (если всё покрыто, напиши, что таких нет).\n"
        )

    prompt = (
        f"{instruction}\n"
        "=== КОНТЕКСТ ===\n"
        f"{context_text}\n"
        "=== ВОПРОС ===\n"
        f"{query}\n"
        "=== ИНСТРУКЦИЯ ПО ФОРМАТУ ВЫВОДА ===\n"
        "Ответ строго в следующей структуре (без лишних разделов):\n"
        "КРАТКИЙ ОТВЕТ:\n"
        "- ...\n\n"
        "ОБОСНОВАНИЕ:\n"
        "- ...\n\n"
        "НЕДОСТАЮЩИЕ ДАННЫЕ:\n"
        "- ...\n\n"
        "=== ОТВЕТ МОДЕЛИ ===\n"
    )
    return prompt'''

def build_rag_prompt(
    query: str,
    context_text: str,
    instruction: str = None,
):
    """
    Собирает финальный текстовый prompt для LLM.
    Ролевая инструкция + жёсткая структура ответа.
    Всё служебное — на английском, ответы тоже на английском.
    """
    if instruction is None:
        instruction = (
        "You are an expert in document analysis and RAG.\n"
        "Your task is to answer questions strictly based on the provided context.\n"
        "Answer in English, clearly and in a structured way.\n"
        "Do not invent facts that are not present in the context.\n"
        "If the information is insufficient, say so explicitly.\n\n"
        "Answer format:\n"
        "1) SHORT ANSWER — 1–3 sentences, the main idea.\n"
        "2) REASONING — detailed explanation referring to key parts of the context (paraphrased).\n"
        "3) MISSING INFORMATION — what exactly cannot be answered from this context.\n"
)

    prompt = (
        f"{instruction}\n"
        "=== CONTEXT ===\n"
        f"{context_text}\n"
        "=== QUESTION ===\n"
        f"{query}\n"
        "=== OUTPUT FORMAT INSTRUCTIONS ===\n"
        "Respond strictly using the following structure (no extra sections):\n"
        "SHORT ANSWER:\n"
        "- ...\n\n"
        "REASONING:\n"
        "- ...\n\n"
        "MISSING INFORMATION:\n"
        "- ...\n\n"
        "=== MODEL ANSWER ===\n"
    )
    return prompt



def answer_question(
    query: str,
    max_new_tokens: int = 256,
    temperature: float = 0.2,
    top_p: float = 0.9,
    debug: bool = False,
):
    """
    Высокоуровневая функция:
    1) ищет релевантные чанки (retrieve_relevant_chunks)
    2) собирает промпт (build_rag_prompt)
    3) генерирует ответ (generate_answer)
    """
    retrieval = retrieve_relevant_chunks(query)
    context_text = retrieval["context_text"]
    selected_chunks = retrieval["chunks"]

    if debug:
        print("=== DEBUG: выбрано чанков ===", len(selected_chunks))
        for ch in selected_chunks:
            print(f"- chunk_id={ch['chunk_id']}, doc_id={ch['doc_id']}, score={ch['score_hybrid']:.4f}")

    if not context_text.strip():
        return (
            "КРАТКИЙ ОТВЕТ:\n"
            "- Я не нашёл релевантной информации в документах.\n\n"
            "ОБОСНОВАНИЕ:\n"
            "- Гибридный поиск не вернул подходящих фрагментов, поэтому я не могу ответить на вопрос на основе контекста.\n\n"
            "НЕДОСТАЮЩИЕ ДАННЫЕ:\n"
            "- Не хватает любых фрагментов документов, относящихся к заданному вопросу."
        )

    prompt = build_rag_prompt(query, context_text)

    answer = generate_answer(
    prompt,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=True,
    )

    return answer.strip()


print("Обновлённые build_rag_prompt() и answer_question() определены.")


Обновлённые build_rag_prompt() и answer_question() определены.


In [84]:
# === Block 7C. Постобработка ответа: normalize_rag_answer() ===

import re

def _extract_section(raw_text: str, current_name_re: str, next_names_re: str):
    """
    Вырезает кусок текста между заголовком current_name_re и
    следующим заголовком из next_names_re (или до конца текста).
    """
    pattern = rf"(?is){current_name_re}\s*:?\s*(.*?)(?:(?:{next_names_re})\s*:|\Z)"
    m = re.search(pattern, raw_text, flags=re.IGNORECASE | re.DOTALL)
    if not m:
        return ""
    content = m.group(1).strip()
    return content


def normalize_rag_answer(raw_text: str) -> str:
    """
    Приводит ответ к виду:

    КРАТКИЙ ОТВЕТ:
    - ...

    ОБОСНОВАНИЕ:
    - ...

    НЕДОСТАЮЩИЕ ДАННЫЕ:
    - ...
    """
    if not isinstance(raw_text, str):
        raw_text = str(raw_text or "")

    text = raw_text.strip()
    if not text:
        return (
            "КРАТКИЙ ОТВЕТ:\n"
            "- Модель не смогла сгенерировать ответ.\n\n"
            "ОБОСНОВАНИЕ:\n"
            "- Ответ модели пуст.\n\n"
            "НЕДОСТАЮЩИЕ ДАННЫЕ:\n"
            "- Не хватает любой информации для ответа."
        )

    # Регэкспы для заголовков (в нижнем регистре, но матчим с IGNORECASE)
    name_short = r"краткий\s+ответ"
    name_reason = r"обоснование"
    name_missing = r"недостающие\s+данные"

    # Попробуем достать три секции из сырого текста
    short_raw = _extract_section(
        text,
        current_name_re=name_short,
        next_names_re=f"{name_reason}|{name_missing}",
    )
    reason_raw = _extract_section(
        text,
        current_name_re=name_reason,
        next_names_re=name_missing,
    )
    missing_raw = _extract_section(
        text,
        current_name_re=name_missing,
        next_names_re=r"$^",  # "ничего", чтобы матчить до конца
    )

    # Если вообще не нашли "краткий ответ", считаем, что весь текст — это ОБОСНОВАНИЕ
    if not short_raw and not reason_raw and not missing_raw:
        reason_raw = text

    def _normalize_section_body(content: str, default_msg: str) -> str:
        content = content.strip()
        if not content:
            return f"- {default_msg}"
        # Если нет ни одной строки, начинающейся с "-", добавим буллет
        lines = [ln.rstrip() for ln in content.splitlines() if ln.strip()]
        if not lines:
            return f"- {default_msg}"
        if not any(ln.lstrip().startswith("-") for ln in lines):
            # склеим в одну строку с одним буллетом
            return "- " + " ".join(lines)
        # иначе вернём как есть (но без лишних пустых строк)
        return "\n".join(lines)

    short_norm = _normalize_section_body(
        short_raw, "Модель не указала краткий ответ."
    )
    reason_norm = _normalize_section_body(
        reason_raw, "Модель не привела обоснование."
    )
    missing_norm = _normalize_section_body(
        missing_raw, "Модель не указала недостающие данные (считаем, что их нет или они не выделены)."
    )

    normalized = (
        "КРАТКИЙ ОТВЕТ:\n"
        f"{short_norm}\n\n"
        "ОБОСНОВАНИЕ:\n"
        f"{reason_norm}\n\n"
        "НЕДОСТАЮЩИЕ ДАННЫЕ:\n"
        f"{missing_norm}"
    )

    return normalized.strip()

print("Функция normalize_rag_answer() определена.")


Функция normalize_rag_answer() определена.


# мини чек как дела

In [85]:
# === Block 8. Ручная проверка пайплайна RAG на 1–2 запросах ===

# ВНИМАНИЕ:
# перед этим блоком должны быть выполнены:
# - Block 0 (импорты, пути, конфиг)
# - Block 1 (поиск документов)
# - Block 2 (загрузка текста)
# - Block 3A, 3B (чанкинг)
# - Block 4A, 4B (эмбеддинги + FAISS)
# - Block 5A, 5B (BM25 + hybrid search)
# - Block 6A, 6B (LLM + generate_answer)
# - Block 7A, 7B (retrieve_relevant_chunks + answer_question)

if "answer_question" not in globals():
    print("Функция answer_question не найдена. Проверь, что все предыдущие блоки выполнены.")
else:
    # пример тестового запроса — подставь сюда что-то из своих документов
    test_queries = [
        "What is the general purpose of this document?",
        "What are the main conclusions presented in the text?",
    ]

    for i, q in enumerate(test_queries, start=1):
        print("=" * 80)
        print(f"[ТЕСТ {i}] ВОПРОС:")
        print(q)
        print("-" * 80)

        try:
            # debug=True, чтобы увидеть, какие чанки выбираются
            answer = answer_question(
                q,
                max_new_tokens=256,
                temperature=0.2,
                top_p=0.9,
                debug=True,
            )
            print("\n[ОТВЕТ]:")
            print(answer)
        except Exception as e:
            print("Ошибка при вызове answer_question:", repr(e))

    print("=" * 80)
    print("Тесты Block 8 выполнены.")


[ТЕСТ 1] ВОПРОС:
What is the general purpose of this document?
--------------------------------------------------------------------------------


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

=== DEBUG: выбрано чанков === 8
- chunk_id=2629, doc_id=0, score=0.8805
- chunk_id=542, doc_id=0, score=0.8740
- chunk_id=3844, doc_id=0, score=0.8531
- chunk_id=2625, doc_id=0, score=0.6000
- chunk_id=44, doc_id=0, score=0.5833
- chunk_id=514, doc_id=0, score=0.5753
- chunk_id=1771, doc_id=0, score=0.5703
- chunk_id=4315, doc_id=0, score=0.5213

[ОТВЕТ]:
SHORT ANSWER:
To provide guidance on how to use the document effectively.

REASONING:
This response explains that the document serves as a guide or reference for readers to understand and utilize the content within it.

MISSING INFORMATION:
None identified in the given context. The document appears to serve as a resource or manual for users to navigate through various topics and concepts related to cognition and learning.Human resources department's role in performance appraisal meetings includes communication of concerns and recognition of positive aspects, aiming to enhance employee performance and decision-making regarding terminat

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

=== DEBUG: выбрано чанков === 8
- chunk_id=567, doc_id=0, score=0.8434
- chunk_id=349, doc_id=0, score=0.6000
- chunk_id=350, doc_id=0, score=0.5992
- chunk_id=465, doc_id=0, score=0.5702
- chunk_id=44, doc_id=0, score=0.5470
- chunk_id=3280, doc_id=0, score=0.5319
- chunk_id=3693, doc_id=0, score=0.5281
- chunk_id=338, doc_id=0, score=0.5247

[ОТВЕТ]:
Short Answer:
The main conclusion presented in the text is that hypotheses are crucial in bridging the gap between ideas and the real world, serving as testable predictions that help refine theories over time.

Reasoning:
The passage emphasizes the importance of hypotheses in scientific inquiry. It states that "hypotheses are extremely important because they bridge the gap between the realm of ideas and the real world" and that "as specific hypotheses are tested, theories are modified and refined to reflect and incorporate the result of these tests." This directly supports the conclusion that hypotheses play a critical role in advancing 

In [86]:
# === Block 8B. Отладка: смотрим контекст и чанки для одного запроса ===

if "retrieve_relevant_chunks" not in globals():
    print("Функция retrieve_relevant_chunks не найдена. Сначала выполни предыдущие блоки.")
else:
    # сюда подставь конкретный интересующий вопрос
    debug_query = "What is the general purpose of this document?"

    print("=" * 80)
    print("[DEBUG] ВОПРОС:")
    print(debug_query)
    print("=" * 80)

    try:
        retrieval = retrieve_relevant_chunks(
            debug_query,
            top_k_faiss=20,
            top_k_bm25=20,
            top_k_final=8,
            alpha_faiss=0.6,
            max_context_chars=MAX_CONTEXT_CHARS,
        )

        selected_chunks = retrieval["chunks"]
        context_text = retrieval["context_text"]

        print(f"[DEBUG] Количество выбранных чанков: {len(selected_chunks)}")
        print("-" * 80)
        print("[DEBUG] Список чанков (chunk_id, doc_id, score):")
        for ch in selected_chunks:
            print(
                f"- chunk_id={ch['chunk_id']}, "
                f"doc_id={ch['doc_id']}, "
                f"score_hybrid={ch['score_hybrid']:.4f}"
            )

        print("=" * 80)
        print("[DEBUG] КОНТЕКСТ, ПЕРЕДАВАЕМЫЙ В МОДЕЛЬ:")
        print(context_text[:2000])  # при желании можно убрать [:2000], чтобы видеть всё
        if len(context_text) > 2000:
            print("\n...[обрезано, полный контекст длиннее 2000 символов]")

        print("=" * 80)
        print("Block 8B: отладка контекста завершена.")

    except Exception as e:
        print("Ошибка в retrieve_relevant_chunks:", repr(e))


[DEBUG] ВОПРОС:
What is the general purpose of this document?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[DEBUG] Количество выбранных чанков: 8
--------------------------------------------------------------------------------
[DEBUG] Список чанков (chunk_id, doc_id, score):
- chunk_id=2629, doc_id=0, score_hybrid=0.8805
- chunk_id=542, doc_id=0, score_hybrid=0.8740
- chunk_id=3844, doc_id=0, score_hybrid=0.8531
- chunk_id=2625, doc_id=0, score_hybrid=0.6000
- chunk_id=44, doc_id=0, score_hybrid=0.5833
- chunk_id=514, doc_id=0, score_hybrid=0.5753
- chunk_id=1771, doc_id=0, score_hybrid=0.5703
- chunk_id=4315, doc_id=0, score_hybrid=0.5213
[DEBUG] КОНТЕКСТ, ПЕРЕДАВАЕМЫЙ В МОДЕЛЬ:
Key Terms
accommodation adjustment of a schema by changing a scheme to accommodate new information different
from what was already known
adolescence period of development that begins at puberty and ends at early adulthood
adrenarche maturing of the adrenal glands
advance directive a written legal document that details specific interventions a person wants (see living will)
assimilation adjustment of a schema by add

# получение вопросов и сабмит

In [87]:
# === Block 9B. Универсальная генерация сабмита в разных форматах (обновлённая версия) ===

import json

# Режимы:
# "simple"      -> id, answer
# "ru_rag"      -> ИДЕНТИФИКАТОР, контекст, отвечать, ссылки
# "debug_full"  -> id, question, context, answer, refs_json
# "fr_rag"      -> id, вопрос, ответ, Контекст, ref_page
SUBMISSION_MODE = "simple"  # "simple" / "ru_rag" / "debug_full" / "fr_rag"

print("SUBMISSION_MODE:", SUBMISSION_MODE)


def rag_answer_with_context_and_refs(
    query: str,
    max_new_tokens: int = 256,
    temperature: float = 0.2,
    top_p: float = 0.9,
):
    """
    Возвращает:
    - answer: строка ответа модели
    - context_text: текст контекста, который ушёл в модель
    - refs_json: JSON-строка с ссылками на использованные чанки/доки (chunk_id, doc_id, score, path)
    - refs_dict: тот же refs в виде dict (на случай, если формату сабмита нужно вытащить что-то одно)
    """
    retrieval = retrieve_relevant_chunks(query)
    context_text = retrieval["context_text"]
    selected_chunks = retrieval["chunks"]

    if not context_text.strip():
        empty_answer = (
            "КРАТКИЙ ОТВЕТ:\n"
            "- Я не нашёл релевантной информации в документах.\n\n"
            "ОБОСНОВАНИЕ:\n"
            "- Гибридный поиск не вернул подходящих фрагментов, поэтому я не могу ответить на вопрос на основе контекста.\n\n"
            "НЕДОСТАЮЩИЕ ДАННЫЕ:\n"
            "- Не хватает любых фрагментов документов, относящихся к заданному вопросу."
        )
        refs_dict = {"chunks": []}
        refs_json = json.dumps(refs_dict, ensure_ascii=False)
        return empty_answer, "", refs_json, refs_dict

    prompt = build_rag_prompt(query, context_text)

    answer = generate_answer(
    prompt,
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=True,
    )

    answer_norm = normalize_rag_answer(answer)

    # добавим в refs не только chunk_id/doc_id/score, но и path, если он есть в chunks_df
    chunk_info_list = []
    for ch in selected_chunks:
        cid = int(ch["chunk_id"])
        did = int(ch["doc_id"])
        score = float(ch["score_hybrid"])

        # ищем строку в chunks_df по chunk_id
        try:
            row_match = chunks_df.loc[chunks_df["chunk_id"] == cid].iloc[0]
            path = str(row_match.get("path", ""))
        except Exception:
            path = ""

        chunk_info_list.append(
            {
                "chunk_id": cid,
                "doc_id": did,
                "score": score,
                "path": path,
            }
        )

    refs_dict = {"chunks": chunk_info_list}
    refs_json = json.dumps(refs_dict, ensure_ascii=False)

    return answer.strip(), context_text, refs_json, refs_dict


SUBMISSION_MODE: simple


In [88]:
# Режимы:
# "simple"            -> id, answer
# "ru_rag"            -> ИДЕНТИФИКАТОР, контекст, отвечать, ссылки
# "debug_full"        -> id, question, context, answer, refs_json
# "fr_rag"            -> id, вопрос, ответ, Контекст, ref_page
# "fr_rag_no_context" -> id, вопрос, ответ, ref_page
# "id_question_answer"-> id, вопрос, ответ
SUBMISSION_MODE = "ru_rag"


In [89]:
# Путь к файлу с вопросами (подправишь под конкретное соревнование)
# === Block 9A. Конфиг и загрузка вопросов (csv/json/jsonl/parquet) ===

QA_INPUT_PATH = DATA_DIR / "/kaggle/input/competition-psycho/queries.json"
QA_INPUT_FORMAT = "json"

QA_ID_COLUMN = "query_id"
QA_QUESTION_COLUMN = "question"

# 👉 имя файла сабмита теперь зависит от SUBMISSION_MODE
# этот блок лучше запускать ПОСЛЕ того, как задан SUBMISSION_MODE
QA_OUTPUT_PATH = OUTPUT_DIR / f"submission_{SUBMISSION_MODE}.csv"
QA_ANSWER_COLUMN = "answer"

print("QA_INPUT_PATH:", QA_INPUT_PATH)
print("QA_INPUT_FORMAT:", QA_INPUT_FORMAT)
print("QA_OUTPUT_PATH:", QA_OUTPUT_PATH)



def load_qa_dataframe(
    path: Path,
    fmt: str,
    id_col: str,
    question_col: str,
) -> pd.DataFrame:
    fmt = fmt.lower()
    if fmt == "csv":
        df = pd.read_csv(path)
    elif fmt == "json":
        df = pd.read_json(path)  # обычный JSON-массив объектов
    elif fmt == "jsonl":
        df = pd.read_json(path, lines=True)  # JSON Lines (по строкам)
    elif fmt == "parquet":
        df = pd.read_parquet(path)
    else:
        raise ValueError(f"Неизвестный формат QA-файла: {fmt}")

    # Проверяем, что нужные колонки есть
    missing = [c for c in [id_col, question_col] if c not in df.columns]
    if missing:
        raise ValueError(
            f"В файле {path} нет нужных колонок: {missing}. "
            f"Доступны колонки: {list(df.columns)}"
        )

    return df


if not QA_INPUT_PATH.exists():
    print("ВНИМАНИЕ: QA_INPUT_PATH не существует:", QA_INPUT_PATH)
    qa_df = None
else:
    qa_df = load_qa_dataframe(
        QA_INPUT_PATH,
        QA_INPUT_FORMAT,
        QA_ID_COLUMN,
        QA_QUESTION_COLUMN,
    )
    print("Загружено вопросов:", len(qa_df))
    print(qa_df[[QA_ID_COLUMN, QA_QUESTION_COLUMN]].head())


QA_INPUT_PATH: /kaggle/input/competition-psycho/queries.json
QA_INPUT_FORMAT: json
QA_OUTPUT_PATH: /kaggle/working/outputs/submission_ru_rag.csv
Загружено вопросов: 50
   query_id                                      question
0         1  What is the scientific method in psychology?
1         2         What are the basic parts of a neuron?
2         3                 What are the stages of sleep?
3         4                 What is operant conditioning?
4         5        What is problem-solving in psychology?


In [90]:
if qa_df is None or len(qa_df) == 0:
    print("qa_df пуст — сначала проверь Block 9A (загрузка вопросов).")
else:
    rows = []

    print("Начинаю генерацию ответов для всех вопросов...")
    for _, row in tqdm(qa_df.iterrows(), total=len(qa_df)):
        q_id = row[QA_ID_COLUMN]
        q_text = str(row[QA_QUESTION_COLUMN])

        try:
            answer, context_text, refs_json, refs_dict = rag_answer_with_context_and_refs(
                q_text,
                max_new_tokens=256,
                temperature=0.2, # советовал 0.8 трайнуть
                top_p=0.9,
            )
        except Exception as e:
            answer = f"Ошибка при генерации ответа: {repr(e)}"
            context_text = ""
            refs_dict = {"error": repr(e)}
            refs_json = json.dumps(refs_dict, ensure_ascii=False)

        if SUBMISSION_MODE == "simple":
            # Вариант 1: базовый (id, answer)
            rows.append(
                {
                    QA_ID_COLUMN: q_id,
                    QA_ANSWER_COLUMN: answer,
                }
            )

        elif SUBMISSION_MODE == "ru_rag":
            # Вариант 2: вариант из русского примера:
            # ИДЕНТИФИКАТОР, контекст, отвечать, ссылки
            rows.append(
                {
                    "ИДЕНТИФИКАТОР": q_id,
                    "контекст": context_text,
                    "отвечать": answer,
                    "ссылки": refs_json,
                }
            )

        elif SUBMISSION_MODE == "debug_full":
            # Вариант 3: расширенный debug-формат
            rows.append(
                {
                    QA_ID_COLUMN: q_id,
                    QA_QUESTION_COLUMN: q_text,
                    "context": context_text,
                    QA_ANSWER_COLUMN: answer,
                    "refs_json": refs_json,
                }
            )

        elif SUBMISSION_MODE == "fr_rag":
            # Вариант 4: полный французский формат:
            # id, вопрос, ответ, Контекст, ref_page
            if refs_dict.get("chunks"):
                first_ref = refs_dict["chunks"][0]
                ref_page = first_ref.get("path", "")
            else:
                ref_page = ""

            rows.append(
                {
                    "id": q_id,
                    "вопрос": q_text,
                    "ответ": answer,
                    "Контекст": context_text,
                    "ref_page": ref_page,
                }
            )

        elif SUBMISSION_MODE == "fr_rag_no_context":
            # Вариант 5: как в твоём примере БЕЗ контекста:
            # id, вопрос, ответ, ref_page
            if refs_dict.get("chunks"):
                first_ref = refs_dict["chunks"][0]
                ref_page = first_ref.get("path", "")
            else:
                ref_page = ""

            rows.append(
                {
                    "id": q_id,
                    "вопрос": q_text,
                    "ответ": answer,
                    "ref_page": ref_page,
                }
            )

        elif SUBMISSION_MODE == "id_question_answer":
            # Вариант 6: только id, вопрос, ответ (без ref_page, без контекста)
            rows.append(
                {
                    "id": q_id,
                    "вопрос": q_text,
                    "ответ": answer,
                }
            )

        else:
            raise ValueError(f"Неизвестный SUBMISSION_MODE: {SUBMISSION_MODE}")



Начинаю генерацию ответов для всех вопросов...


  0%|          | 0/50 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  2%|▏         | 1/50 [00:08<07:00,  8.57s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

 96%|█████████▌| 48/50 [06:51<00:17,  8.59s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 98%|█████████▊| 49/50 [07:00<00:08,  8.57s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 50/50 [07:08<00:00,  8.57s/it]


In [91]:
# === Собираем submission_df из уже посчитанных rows ===

if "rows" not in globals():
    raise RuntimeError("Переменная rows не найдена. Нужно ещё раз запустить ячейку с генерацией (где rows.append(...)).")

print("Количество строк в rows:", len(rows))

submission_df = pd.DataFrame(rows)
print("Колонки submission_df:", list(submission_df.columns))
print(submission_df.head())


Количество строк в rows: 50
Колонки submission_df: ['ИДЕНТИФИКАТОР', 'контекст', 'отвечать', 'ссылки']
   ИДЕНТИФИКАТОР                                           контекст  \
0              1  increasingly match the larger population, and ...   
1              2  the other hand, serve as interconnected inform...   
2              3  in amplitude. The first stage of NREM sleep is...   
3              4  a. structuralism\nb. functionalism\nc. Gestalt...   
4              5  how language may influence cognition remains a...   

                                            отвечать  \
0  SHORT ANSWER:\nThe scientific method in psycho...   
1  SHORT ANSWER:\nA neuron consists of three main...   
2  SHORT ANSWER:\nThe stages of sleep include:\n\...   
3  SHORT ANSWER:\nOperant conditioning is a form ...   
4  SHORT ANSWER:\nProblem-solving in psychology r...   

                                              ссылки  
0  {"chunks": [{"chunk_id": 338, "doc_id": 0, "sc...  
1  {"chunks": [{"chunk_

In [51]:
submission_df = pd.DataFrame(rows)
submission_df.to_csv(QA_OUTPUT_PATH, index=False)

In [50]:
'''# === Финальный сабмит: id,answer ===

if "submission_df" not in globals():
    raise RuntimeError("submission_df не найден. Сначала собери его из rows (см. предыдущую ячейку).")

print("Текущие колонки:", list(submission_df.columns))

# если твой simple-режим уже делает 'id' и 'answer', то всё просто:
if "id" in submission_df.columns and "answer" in submission_df.columns:
    simple_sub = submission_df[["id", "answer"]].copy()
else:
    # если id-колонка называется иначе (например, QA_ID_COLUMN / 'query_id'):
    id_col = QA_ID_COLUMN  # у тебя он уже задан в Block 9A
    simple_sub = submission_df[[id_col, QA_ANSWER_COLUMN]].rename(
        columns={id_col: "id", QA_ANSWER_COLUMN: "answer"}
    )

final_path = OUTPUT_DIR / "submission.csv"
simple_sub.to_csv(final_path, index=False)

print("Сабмит сохранён в:", final_path)
print(simple_sub.head())'''


'# === Финальный сабмит: id,answer ===\n\nif "submission_df" not in globals():\n    raise RuntimeError("submission_df не найден. Сначала собери его из rows (см. предыдущую ячейку).")\n\nprint("Текущие колонки:", list(submission_df.columns))\n\n# если твой simple-режим уже делает \'id\' и \'answer\', то всё просто:\nif "id" in submission_df.columns and "answer" in submission_df.columns:\n    simple_sub = submission_df[["id", "answer"]].copy()\nelse:\n    # если id-колонка называется иначе (например, QA_ID_COLUMN / \'query_id\'):\n    id_col = QA_ID_COLUMN  # у тебя он уже задан в Block 9A\n    simple_sub = submission_df[[id_col, QA_ANSWER_COLUMN]].rename(\n        columns={id_col: "id", QA_ANSWER_COLUMN: "answer"}\n    )\n\nfinal_path = OUTPUT_DIR / "submission.csv"\nsimple_sub.to_csv(final_path, index=False)\n\nprint("Сабмит сохранён в:", final_path)\nprint(simple_sub.head())'

In [ ]:
'''# === Финальный сабмит из submission_df в формате id,answer ===

if "submission_df" not in globals():
    raise RuntimeError("submission_df не найден. Сначала собери его из rows.")

print("Текущие колонки:", list(submission_df.columns))

# Прямо жёстко переименовываем русские имена в нужные:
simple_sub = submission_df.rename(
    columns={
        "ИДЕНТИФИКАТОР": "id",
        "отвечать": "answer",
    }
)[["id", "answer"]]  # берём только эти две колонки и в нужном порядке

final_path = OUTPUT_DIR / "submission.csv"
simple_sub.to_csv(final_path, index=False)

print("Сабмит сохранён в:", final_path)
print(simple_sub.head())'''


In [ ]:
'''# === Block 9. Формирование сабмита для competition-psycho ===

import json
from pathlib import Path

# путь к файлу с вопросами
QA_INPUT_PATH = Path("/kaggle/input/competition-psycho/queries.json")

with open(QA_INPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

qa_df = pd.DataFrame(data)
print("Пример строк из queries.json:")
print(qa_df.head())

# ВАЖНО: подправь эти два имени колонок, если они другие в queries.json
ID_COL = "query_id"       # например: "id" или "query_id"
QUESTION_COL = "question"   # например: "query" или "question"

# проверим, что такие колонки есть
assert ID_COL in qa_df.columns, f"В qa_df нет колонки {ID_COL}, есть: {list(qa_df.columns)}"
assert QUESTION_COL in qa_df.columns, f"В qa_df нет колонки {QUESTION_COL}, есть: {list(qa_df.columns)}"

rows = []

print("Генерирую ответы для всех вопросов...")
for _, row in tqdm(qa_df.iterrows(), total=len(qa_df)):
    q_id = row[ID_COL]
    q_text = str(row[QUESTION_COL])

    ans = answer_question(
        q_text,
        max_new_tokens=256,
        temperature=0.2,
        top_p=0.9,
        debug=False,
    )

    rows.append(
        {
            "id": q_id,
            "answer": ans,
        }
    )

submission_df = pd.DataFrame(rows)

# файл сабмита
sub_path = OUTPUT_DIR / "submission.csv"
submission_df.to_csv(sub_path, index=False)

print("Сабмит сохранён в:", sub_path)
print(submission_df.head())'''


если колонки в другом порядке:

In [92]:
print(submission_df.columns)
# Index(['id', 'вопрос', 'ответ', 'Контекст', 'ref_page'], dtype='object')


Index(['ИДЕНТИФИКАТОР', 'контекст', 'отвечать', 'ссылки'], dtype='object')


In [ ]:
desired_cols = ["id", "вопрос", "Контекст", "ответ"]
submission_df = submission_df[desired_cols]


Главная идея:
сначала формируешь submission_df,
потом (перед to_csv) один раз явно задаёшь порядок колонок:

In [ ]:
submission_df = submission_df[["id", "вопрос", "контекст", "ответ"]]
submission_df.to_csv(QA_OUTPUT_PATH, index=False)


# прочие нюансы формирования сабмита

## Шпаргалка по колонкам в разных режимах

SUBMISSION_MODE = "simple"

In [ ]:
submission_df.columns == [QA_ID_COLUMN, QA_ANSWER_COLUMN]
# например: ['id', 'answer']

или

In [ ]:
submission_df = submission_df[[QA_ID_COLUMN, QA_ANSWER_COLUMN]]
submission_df.rename(columns={QA_ID_COLUMN: "id", QA_ANSWER_COLUMN: "answer"}, inplace=True)


SUBMISSION_MODE = "debug_full"

In [ ]:
[QA_ID_COLUMN, QA_QUESTION_COLUMN, "context", QA_ANSWER_COLUMN, "refs_json"]
# например: ['id', 'question', 'context', 'answer', 'refs_json']


SUBMISSION_MODE = "fr_rag"

In [ ]:
['id', 'вопрос', 'ответ', 'Контекст', 'ref_page']

SUBMISSION_MODE = "fr_rag_no_context"

In [ ]:
['id', 'вопрос', 'ответ', 'ref_page']


SUBMISSION_MODE = "id_question_answer"

In [ ]:
['id', 'вопрос', 'ответ']


Универсальный мини-шаблон для «любого конкурса»

In [ ]:
desired_cols = ["id", "question", "answer"]  # подставляешь нужный порядок/имена
submission_custom = submission_df[desired_cols].copy()
submission_custom.to_csv(OUTPUT_DIR / "submission_custom.csv", index=False)


## переименовать колонки

Переименовать несколько колонок по словарю

In [93]:
submission_df = submission_df.rename(
    columns={
        "ИДЕНТИФИКАТОР": "ID",
        "отвечать": "answer",
        "контекст": "context",
        "ссылки": "references",
    }
)

print(submission_df.columns)
# Index(['id', 'context', 'answer', 'refs_json'], dtype='object')



Index(['ID', 'context', 'answer', 'references'], dtype='object')


In [94]:
submission_df.to_csv(QA_OUTPUT_PATH, index=False)

In [98]:
submission_df

,ID,context,answer,references
0,1,"increasingly match the larger population, and ...",SHORT ANSWER:\nThe scientific method in psycho...,"{""chunks"": [{""chunk_id"": 338, ""doc_id"": 0, ""sc..."
1,2,"the other hand, serve as interconnected inform...",SHORT ANSWER:\nA neuron consists of three main...,"{""chunks"": [{""chunk_id"": 688, ""doc_id"": 0, ""sc..."
2,3,in amplitude. The first stage of NREM sleep is...,SHORT ANSWER:\nThe stages of sleep include:\n\...,"{""chunks"": [{""chunk_id"": 973, ""doc_id"": 0, ""sc..."
3,4,a. structuralism\nb. functionalism\nc. Gestalt...,SHORT ANSWER:\nOperant conditioning is a form ...,"{""chunks"": [{""chunk_id"": 288, ""doc_id"": 0, ""sc..."
4,5,how language may influence cognition remains a...,SHORT ANSWER:\nProblem-solving in psychology r...,"{""chunks"": [{""chunk_id"": 1996, ""doc_id"": 0, ""s..."
5,6,8.1 How Memory Functions\nLEARNING OBJECTIVES\...,SHORT ANSWER:\nThe three stages of memory are ...,"{""chunks"": [{""chunk_id"": 2023, ""doc_id"": 0, ""s..."
6,7,"with similar circumstances. Over time, several...",SHORT ANSWER:\nThe key components of emotion i...,"{""chunks"": [{""chunk_id"": 2860, ""doc_id"": 0, ""s..."
7,8,given situation. Personality traits are relati...,"SHORT ANSWER:\nOpenness, Conscientiousness, Ex...","{""chunks"": [{""chunk_id"": 213, ""doc_id"": 0, ""sc..."
8,9,Summary\n12.1 What Is Social Psychology?\nSoci...,SHORT ANSWER:\nSocial psychology is the study ...,"{""chunks"": [{""chunk_id"": 3666, ""doc_id"": 0, ""s..."
9,10,16.5 The Sociocultural Model and Therapy Utili...,SHORT ANSWER:\nThe sociocultural model in ther...,"{""chunks"": [{""chunk_id"": 5363, ""doc_id"": 0, ""s..."


Переименовать только одну колонку

In [ ]:
submission_df = submission_df.rename(columns={"ИДЕНТИФИКАТОР": "id"})


Полностью задать новый список имён (если порядок уже правильный)

In [ ]:
submission_df.columns
# ['ИДЕНТИФИКАТОР', 'контекст', 'отвечать', 'ссылки']


и ты хочешь точно такой же порядок, но другие названия:

In [ ]:
submission_df.columns = ["id", "context", "answer", "refs_json"]

# пытаюсь улучшить скор

## cross-encoder reranker 

In [78]:
# === Block 10A. Cross-encoder reranker (HF) ===

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Небольшой, но сильный cross-encoder, обученный на MS MARCO (английский)
RERANKER_MODEL_ID = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# Используем тот же GPU, что и для LLM, если доступен
RERANKER_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Loading cross-encoder reranker:", RERANKER_MODEL_ID)
reranker_tokenizer = AutoTokenizer.from_pretrained(RERANKER_MODEL_ID)
reranker_model = AutoModelForSequenceClassification.from_pretrained(RERANKER_MODEL_ID).to(RERANKER_DEVICE)

# Проверим, что у модели один выходной логит (score)
num_labels = reranker_model.config.num_labels
print(f"Reranker loaded on {RERANKER_DEVICE}, num_labels={num_labels}")


Loading cross-encoder reranker: cross-encoder/ms-marco-MiniLM-L-6-v2


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Reranker loaded on cuda, num_labels=1


In [79]:
# === Block 10B. Rerank кандидатов cross-encoder'ом + обновлённый retrieve_relevant_chunks ===

import numpy as np

# Можно будет крутить эти параметры
CE_TOP_K_CANDIDATES = 50   # сколько кандидатов берём после hybrid для rerank
CE_TOP_K_FINAL = 8         # сколько чанкoв остаётся после rerank (пойдут в контекст)

def rerank_chunks_with_ce(query: str, candidates: list, top_k_final: int = CE_TOP_K_FINAL):
    """
    Переранжирует список кандидатов (dict'ы с chunk_id/doc_id/score_hybrid)
    с помощью cross-encoder'а.

    candidates: список словарей, у которых есть хотя бы 'chunk_id' и 'doc_id'.
    Возвращает НОВЫЙ список кандидатов, отсортированный по score_ce (убывание).
    """
    if not candidates:
        return []

    # Ограничим количество кандидатов сверху (чтобы не грузить лишнее)
    cand = candidates[:CE_TOP_K_CANDIDATES]

    # Собираем пары (query, chunk_text)
    pair_texts = []
    for ch in cand:
        cid = int(ch["chunk_id"])
        # достаём текст чанка из chunks_df
        try:
            row_match = chunks_df.loc[chunks_df["chunk_id"] == cid].iloc[0]
            chunk_text = str(row_match.get("chunk_text", ""))
        except Exception:
            chunk_text = ""
        pair_texts.append((query, chunk_text))

    # Токенизируем пачкой
    inputs = reranker_tokenizer(
        [p[0] for p in pair_texts],
        [p[1] for p in pair_texts],
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    ).to(RERANKER_DEVICE)

    # Считаем скор для каждого (query, chunk)
    with torch.no_grad():
        logits = reranker_model(**inputs).logits

    # Если num_labels=1, logits.shape = [batch, 1], иначе берём логит класса 1
    if logits.shape[1] == 1:
        scores = logits.squeeze(-1).detach().cpu().numpy()
    else:
        # предполагаем, что релевантность = логит класса 1
        scores = logits[:, 1].detach().cpu().numpy()

    # Добавляем score_ce к кандидатам
    for ch, sc in zip(cand, scores):
        ch["score_ce"] = float(sc)

    # Сортируем по score_ce (desc)
    cand_sorted = sorted(cand, key=lambda x: x.get("score_ce", 0.0), reverse=True)

    # Оставляем top_k_final (можно потом крутить)
    return cand_sorted[:top_k_final]


def retrieve_relevant_chunks(
    query: str,
    top_k_faiss: int = 50,
    top_k_bm25: int = 50,
    top_k_final: int = CE_TOP_K_FINAL,
    alpha_faiss: float = 0.6,
    max_context_chars: int = MAX_CONTEXT_CHARS,
):
    """
    Обновлённая версия:
    1) Гибридный поиск (FAISS + BM25) → кандидаты с score_hybrid
    2) Cross-encoder reranker (MS MARCO) → score_ce, сортировка
    3) Берём top_k_final чанков, собираем контекст

    Возвращает:
    - dict с полями:
        - "chunks": список выбранных чанков (dict)
        - "context_text": текст, который пойдёт в LLM
    """
    # --- 1. Гибридный поиск по эмбеддингам и BM25 ---
    hybrid_candidates = hybrid_search_chunks(
        query=query,
        top_k_faiss=top_k_faiss,
        top_k_bm25=top_k_bm25,
        alpha_faiss=alpha_faiss,
    )

    if not hybrid_candidates:
        return {"chunks": [], "context_text": ""}

    # --- 2. Rerank candidates cross-encoder'ом ---
    reranked = rerank_chunks_with_ce(query, hybrid_candidates, top_k_final=top_k_final)

    # --- 3. Собираем context_text из reranked чанков ---
    # Заодно избегаем дублей по chunk_id
    seen_chunk_ids = set()
    selected_chunks = []
    context_parts = []

    for ch in reranked:
        cid = int(ch["chunk_id"])
        if cid in seen_chunk_ids:
            continue
        seen_chunk_ids.add(cid)

        # достаём текст чанка
        row_match = chunks_df.loc[chunks_df["chunk_id"] == cid].iloc[0]
        chunk_text = str(row_match.get("chunk_text", ""))

        selected_chunks.append(ch)
        context_parts.append(chunk_text)

    # Склеиваем контекст, при необходимости обрезаем по длине
    context_text = "\n\n".join(context_parts)
    if len(context_text) > max_context_chars:
        context_text = context_text[:max_context_chars]

    return {
        "chunks": selected_chunks,
        "context_text": context_text,
    }

print("Cross-encoder reranker и обновлённый retrieve_relevant_chunks() определены.")


Cross-encoder reranker и обновлённый retrieve_relevant_chunks() определены.


перезапустить (в каком порядке теперь это должно быть):


Block 0 (пути, константы).

Block 0.5 (копирование book.pdf и queries.json — если есть).

Block 1 (список документов).

Block 2 (чтение PDF → raw текст).

Block 3A / 3B (чанкинг → chunks_df).

Block 4A / 4B (эмбеддинги + FAISS-индекс).

Block 5A / 5B (BM25 + hybrid_search_chunks).

Block 10A (загрузка cross-encoder reranker).

Block 10B (rerank + новый retrieve_relevant_chunks).

Block 6A / 6B (LLM + generate_answer).

Block 7B (build_rag_prompt, answer_question — уже будет использовать новый retrieve_relevant_chunks).

Block 8 / 8B (ручная проверка).

Block 9A / 9B (загрузка queries.json → генерация сабмита).

## Multi-query + Step-back generation (LLM)

In [ ]:
# === Block 11A. Multi-query generation (paraphrases + step-back) ===

def generate_search_queries(
    question: str,
    num_paraphrases: int = 3,
    add_step_back: bool = True,
    max_new_tokens: int = 128,
):
    """
    Генерирует несколько альтернативных поисковых запросов (multi-query)
    + step-back (более общий вопрос).
    """

    # 1) Запрос на перефразировки
    para_prompt = (
        "Rewrite the question into several diverse search queries that can help "
        "retrieve relevant passages from a textbook.\n"
        f"Original question: {question}\n\n"
        f"Generate {num_paraphrases} short search queries in English.\n"
        "Return them as a numbered list. No extra text."
    )

    para_output = gen_pipe(
        para_prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )[0]["generated_text"]

    # извлечём строки с запросами
    paraphrases = []
    for line in para_output.splitlines():
        line = line.strip()
        if line and (line[0].isdigit() or line.startswith("-")):
            q = line.split(".", 1)[-1].strip()
            if len(q) > 0:
                paraphrases.append(q)

    # fallback если модель плохо отформатировала
    paraphrases = paraphrases[:num_paraphrases]

    # 2) step-back запрос
    step_back_q = None
    if add_step_back:
        sb_prompt = (
            "Rewrite the question into a broader, more general version that still helps "
            "retrieve relevant parts of a textbook.\n"
            f"Original question: {question}\n\n"
            "Return only ONE short broader question in English."
        )

        sb_output = gen_pipe(
            sb_prompt,
            max_new_tokens=80,
            do_sample=True,
            temperature=0.5,
            top_p=0.9,
        )[0]["generated_text"]

        # просто берём первую полноценную строку
        sb_line = sb_output.strip().splitlines()[0]
        step_back_q = sb_line.strip()

    # финальный список запросов
    queries = [question]
    for q in paraphrases:
        if q and q not in queries:
            queries.append(q)

    if step_back_q and step_back_q not in queries:
        queries.append(step_back_q)

    return queries


In [ ]:
# === Block 11B. Multi-query retrieval wrapper ===

def get_candidates_multiquery(
    question: str,
    top_k_faiss: int = 50,
    top_k_bm25: int = 50,
    alpha_faiss: float = 0.6,
):
    """
    Генерирует несколько альтернативных поисковых запросов (multi-query),
    делает гибридный поиск по каждому и объединяет кандидатов.
    """

    queries = generate_search_queries(question)

    all_candidates = []
    for q in queries:
        try:
            cand = hybrid_search_chunks(
                query=q,
                top_k_faiss=top_k_faiss,
                top_k_bm25=top_k_bm25,
                alpha_faiss=alpha_faiss,
            )
            all_candidates.extend(cand)
        except Exception as e:
            print(f"[WARN] hybrid search failed for query '{q}': {e}")

    # deduplicate by chunk_id (оставляем лучший score_hybrid)
    merged = {}
    for ch in all_candidates:
        cid = int(ch["chunk_id"])
        if cid not in merged or ch["score_hybrid"] > merged[cid]["score_hybrid"]:
            merged[cid] = ch

    candidates = list(merged.values())
    # сортируем по score_hybrid как первичному
    candidates = sorted(candidates, key=lambda x: x["score_hybrid"], reverse=True)

    return candidates


In [ ]:
# === Block 11C. Final retrieve_relevant_chunks (multi-query + reranker + adaptive K) ===

ADAPTIVE_THRESHOLD = 0.0     # порог уверенности cross-encoder
MIN_CHUNKS = 3
MAX_CHUNKS = 8

def retrieve_relevant_chunks(
    query: str,
    top_k_faiss: int = 50,
    top_k_bm25: int = 50,
    alpha_faiss: float = 0.6,
    max_context_chars: int = MAX_CONTEXT_CHARS,
):
    """
    Улучшенная версия:
    1. Генерация multi-query (paraphrases + step-back)
    2. Hybrid search (FAISS + BM25) для каждого запроса
    3. Дедупликация кандидатов
    4. Cross-encoder reranking
    5. Adaptive K (по уверенности, а не фиксированному количеству)
    6. Формирование контекста
    """

    # 1. Multi-query candidates
    multi_candidates = get_candidates_multiquery(
        question=query,
        top_k_faiss=top_k_faiss,
        top_k_bm25=top_k_bm25,
        alpha_faiss=alpha_faiss,
    )

    if not multi_candidates:
        return {"chunks": [], "context_text": ""}

    # 2. Cross-encoder rerank
    reranked = rerank_chunks_with_ce(
        query=query,
        candidates=multi_candidates,
        top_k_final=MAX_CHUNKS * 3,   # временно берём много, потом фильтруем
    )

    # 3. Adaptive K
    confident = [ch for ch in reranked if ch.get("score_ce", 0) >= ADAPTIVE_THRESHOLD]

    if len(confident) < MIN_CHUNKS:
        selected = reranked[:MIN_CHUNKS]
    else:
        selected = confident[:MAX_CHUNKS]

    # 4. Формируем context_text
    seen = set()
    parts = []
    final_chunks = []

    for ch in selected:
        cid = int(ch["chunk_id"])
        if cid in seen:
            continue
        seen.add(cid)

        row = chunks_df.loc[chunks_df["chunk_id"] == cid].iloc[0]
        text = str(row["chunk_text"])

        final_chunks.append(ch)
        parts.append(text)

    context_text = "\n\n".join(parts)
    if len(context_text) > max_context_chars:
        context_text = context_text[:max_context_chars]

    return {
        "chunks": final_chunks,
        "context_text": context_text,
    }

print("MULTI-QUERY + RERANKER + ADAPTIVE-K retrieve_relevant_chunks loaded.")


ПЕРВЫЙ ПРОХОД (полностью после изменения кода):

Block 0 — импорты, пути

Block 0.5 — копирование данных (если есть)

Block 1 — список документов

Block 2 — PDF → raw text

Block 3A / 3B — чанкинг → chunks_df

Block 4A / 4B — Embeddings → FAISS

Block 5A / 5B — BM25 → hybrid_search_chunks

Block 6A — LLM загрузка (gen_pipe)

Block 10A — загрузка cross-encoder (reranker)

Block 10B — определение rerank_chunks_with_ce

Block 7B — build_rag_prompt, answer_question

Block 11A — multi-query generation

Block 11B — multiquery candidate gatherer

Block 11C — замена retrieve_relevant_chunks

Block 8 / 8B — тестирование

Block 9A / 9B — генерация submission

ПРИ ПОВТОРНОМ ЗАПУСКЕ (если ядро не перезагружалось):

Можно запускать только:

Block 10A

Block 10B

Block 11A

Block 11B

Block 11C

Block 7B

Block 8 или Block 9

## Self-RAG helper (draft + verify)

In [ ]:
# === Block 12A. Self-RAG helper: draft + verify ===

def build_verify_prompt(question: str, context_text: str, draft_answer: str) -> str:
    """
    Промпт для второго шага Self-RAG:
    проверка и переписывание ответа строго по контексту.
    """
    prompt = (
        "You are a verification and refinement module for a RAG system.\n"
        "Your job is to check whether the draft answer is FULLY supported by the provided context.\n"
        "If any part of the answer is not supported, you must either remove it or rephrase it so that it is strictly grounded in the context.\n"
        "If the context does not contain enough information to answer the question, you must explicitly say that.\n\n"
        "=== CONTEXT ===\n"
        f"{context_text}\n\n"
        "=== QUESTION ===\n"
        f"{question}\n\n"
        "=== DRAFT ANSWER ===\n"
        f"{draft_answer}\n\n"
        "=== TASK ===\n"
        "1) Identify unsupported or speculative parts.\n"
        "2) Rewrite the answer so that EVERY statement is supported by the context.\n"
        "3) Keep the answer clear and concise.\n"
        "4) Answer in English.\n\n"
        "Return ONLY the final improved answer, without any meta-comments.\n"
    )
    return prompt


def self_rag_answer(
    question: str,
    context_text: str,
    max_new_tokens: int = 256,
    temperature_draft: float = 0.4,
    temperature_verify: float = 0.2,
    top_p: float = 0.9,
):
    """
    Двухшаговый ответ:
    1) draft-ответ по стандартному RAG-промпту
    2) проверка и переписывание ответа строго по контексту (Self-RAG)
    """

    # --- Шаг 1. Черновой ответ ---
    draft_prompt = build_rag_prompt(
        query=question,
        context_text=context_text,
    )
    draft_answer = generate_answer(
        prompt=draft_prompt,
        max_new_tokens=max_new_tokens,
        temperature=temperature_draft,
        top_p=top_p,
        do_sample=True,
    )

    # --- Шаг 2. Верификация и улучшение ---
    verify_prompt = build_verify_prompt(
        question=question,
        context_text=context_text,
        draft_answer=draft_answer,
    )
    final_answer = generate_answer(
        prompt=verify_prompt,
        max_new_tokens=max_new_tokens,
        temperature=temperature_verify,
        top_p=top_p,
        do_sample=False,  # на верификации можно не сэмплить
    )

    return final_answer.strip(), draft_answer.strip()


In [ ]:
# === Block 12B. Обновлённый answer_question() с Self-RAG ===

def answer_question(
    question: str,
    max_new_tokens: int = 256,
    temperature_draft: float = 0.4,
    temperature_verify: float = 0.2,
    top_p: float = 0.9,
    debug: bool = True,
):
    """
    Высокоуровневая функция:
    1) достаёт релевантные чанки (retrieve_relevant_chunks)
    2) собирает context_text
    3) делает двухшаговый Self-RAG:
       - draft ответ по стандартному промпту
       - verify+refine ответ строго по контексту
    """

    retrieval = retrieve_relevant_chunks(
        query=question,
        top_k_faiss=50,
        top_k_bm25=50,
        alpha_faiss=0.6,
        max_context_chars=MAX_CONTEXT_CHARS,
    )
    chunks = retrieval.get("chunks", [])
    context_text = retrieval.get("context_text", "")

    if debug:
        print("=== DEBUG: выбрано чанков ===", len(chunks))
        for ch in chunks:
            cid = ch.get("chunk_id")
            did = ch.get("doc_id")
            s_h = ch.get("score_hybrid", None)
            s_ce = ch.get("score_ce", None)
            line = f"- chunk_id={cid}, doc_id={did}, hybrid={s_h:.4f}" if s_h is not None else f"- chunk_id={cid}, doc_id={did}"
            if s_ce is not None:
                line += f", ce={s_ce:.4f}"
            print(line)
        print()

    final_answer, draft_answer = self_rag_answer(
        question=question,
        context_text=context_text,
        max_new_tokens=max_new_tokens,
        temperature_draft=temperature_draft,
        temperature_verify=temperature_verify,
        top_p=top_p,
    )

    if debug:
        print("[DRAFT ANSWER]:")
        print(draft_answer)
        print("\n[FINAL ANSWER]:")
        print(final_answer)

    return final_answer


In [ ]:
# === Block 12C. Обновлённый rag_answer_with_context_and_refs() с Self-RAG ===

def rag_answer_with_context_and_refs(
    question: str,
    max_new_tokens: int = 256,
    temperature_draft: float = 0.4,
    temperature_verify: float = 0.2,
    top_p: float = 0.9,
):
    """
    Расширенная версия ответа для сабмитов:
    - делает retrieve_relevant_chunks (multi-query + reranker)
    - считает final_answer через Self-RAG (draft + verify)
    - возвращает:
        answer, context_text, refs_json, refs_dict
    """

    retrieval = retrieve_relevant_chunks(
        query=question,
        top_k_faiss=50,
        top_k_bm25=50,
        alpha_faiss=0.6,
        max_context_chars=MAX_CONTEXT_CHARS,
    )
    chunks = retrieval.get("chunks", [])
    context_text = retrieval.get("context_text", "")

    # Self-RAG
    final_answer, draft_answer = self_rag_answer(
        question=question,
        context_text=context_text,
        max_new_tokens=max_new_tokens,
        temperature_draft=temperature_draft,
        temperature_verify=temperature_verify,
        top_p=top_p,
    )

    # Собираем refs_dict
    refs = {"chunks": []}
    for ch in chunks:
        cid = int(ch.get("chunk_id"))
        row = chunks_df.loc[chunks_df["chunk_id"] == cid].iloc[0]

        ref_item = {
            "chunk_id": cid,
            "doc_id": int(ch.get("doc_id", row.get("doc_id", 0))),
            "path": str(row.get("path", "")),
            "score_faiss": float(ch.get("score_faiss", 0.0)),
            "score_bm25": float(ch.get("score_bm25", 0.0)),
            "score_hybrid": float(ch.get("score_hybrid", 0.0)),
        }
        if "score_ce" in ch:
            ref_item["score_ce"] = float(ch["score_ce"])

        # если у тебя в chunks_df есть 'page' или 'page_start' — можно добавить сюда:
        if "page" in row.index:
            ref_item["page"] = int(row["page"])

        refs["chunks"].append(ref_item)

    refs_json = json.dumps(refs, ensure_ascii=False)

    return final_answer.strip(), context_text, refs_json, refs


Block 12A

Block 12B

Block 12C

и дальше:

Block 8 — для проверки качества ответов

Block 9B — для нового сабмита

## к слову о параметрах

In [ ]:
'''1. Retrieval Parameters (FAISS + BM25 + Multi-Query + Reranker)

Эти параметры находятся в:

Block 10B — Cross-encoder reranker

Block 11A — Multi-query generation

Block 11B — Candidate merging

Block 11C — Final retrieve_relevant_chunks

🔹 1.1. Adaptive K (Block 11C)
ADAPTIVE_THRESHOLD = 0.0
MIN_CHUNKS = 3
MAX_CHUNKS = 8


ADAPTIVE_THRESHOLD: порог по cross-encoder score.

↑ больше (0.2–0.5) → меньше мусора

↓ меньше (0.0) → шире покрытие

MIN_CHUNKS: минимум чанков.

увеличить до 4 → если LLM слишком поверхностна

уменьшить до 2 → если слишком многословно

MAX_CHUNKS: максимум чанков.

увеличить до 10–12 → лучше покрытие

уменьшить до 6 → меньше шума

🔹 1.2. Hybrid Search Parameters (Block 11B)
top_k_faiss = 50
top_k_bm25 = 50
alpha_faiss = 0.6


top_k_faiss, top_k_bm25:

↑ до 80–100 → больше кандидатов для reranker

↓ до 30 → быстрее, но хуже качество

alpha_faiss:

ближе к 0.7–0.8 → лучше семантический поиск

ближе к 0.4–0.5 → лучше по ключевым словам (если вопросы прямые)

🔹 1.3. Reranker Parameters (Block 10B)
CE_TOP_K_CANDIDATES = 50
CE_TOP_K_FINAL = 8


CE_TOP_K_CANDIDATES: сколько кандидатов проверяет cross-encoder

↑ до 80–100 → выше качество поиска

↓ до 30 → быстрее

CE_TOP_K_FINAL: сколько берём после reranker до adaptive K

обычно от 8 до 16.

🔹 1.4. Multi-Query Generation Parameters (Block 11A)
num_paraphrases = 3
temperature (paraphrases) = 0.7
temperature (step-back) = 0.5


num_paraphrases:

↑ до 4–5 → более устойчивый поиск

↓ до 2 → быстрее

Температуры:

↑ → разнообразные формулировки

↓ → более точные формулировки

#️⃣ 2. Self-RAG Parameters (Block 12A / 12B / 12C)

Эти параметры управляют качеством финального ответа.

🔹 2.1. Draft generation
temperature_draft = 0.4
top_p = 0.9


↑ до 0.5–0.6 → ответы богаче и разнообразнее

↓ до 0.2–0.3 → меньше галлюцинаций, строже привязка к контексту

🔹 2.2. Verification stage
temperature_verify = 0.2


не стоит сильно поднимать — это «строгая» стадия

↑ до 0.3–0.4 → чуть плавнее формулировки

↓ до 0.1 → максимально строгий ответ «по контексту»

🔹 2.3. Token Limits
max_new_tokens = 256


↑ до 320–384 → если ответы обрезаются

↓ до 192 → если модель многословна

#️⃣ 3. Prompting Parameters (Block 7B + Block 12A)
🔹 3.1. Output Structure Strictness

Можно усилить:

более строгие инструкции

указание «не использовать информацию вне контекста»

требование ссылок (pages / sections)

🔹 3.2. Chain-of-Thought

Можно включить короткий CoT:

Think step-by-step, but keep reasoning concise (no more than 3–4 steps).


в draft-промпт.

#️⃣ 4. Chunking & Context Parameters (Block 3A/3B)
🔹 4.1. Overlap

↑ overlap до 40–50% → больше связности

↓ до 20% → меньше шума, короче индекс

🔹 4.2. Max context length

В блоке retrieve_relevant_chunks:

max_context_chars = MAX_CONTEXT_CHARS


↑ до 50k → больше контекста (если модель тянет)

↓ до 20k → если ответы слишком загромождены

#️⃣ 5. Что влияет сильнее всего (приоритет)
🥇 ТОП-5 ПАРАМЕТРОВ, которые дают реальный прирост:

ADAPTIVE_THRESHOLD

MAX_CHUNKS / MIN_CHUNKS

num_paraphrases (multi-query)

top_k_faiss / top_k_bm25

temperature_draft

🥈 Средней важности:

alpha_faiss

temperatures при multi-query

chunk overlap

CE_TOP_K_CANDIDATES

🥉 Наименее критичные:

temperature_verify

top_p

max_new_tokens'''

## новая версия build_verify_prompt с указанием источников

In [ ]:
# === Block 13A. Обновлённый build_verify_prompt с указанием источников ===

def build_verify_prompt(question: str, context_text: str, draft_answer: str) -> str:
    """
    Промпт для второго шага Self-RAG:
    проверка и переписывание ответа строго по контексту + указание источников.
    """

    prompt = (
        "You are a verification and refinement module for a RAG system.\n"
        "Your task is to check whether the DRAFT ANSWER is fully supported by the CONTEXT.\n"
        "If any part of the answer is not supported, you must remove it or rewrite it so that it is strictly grounded in the context.\n"
        "If the context does not contain enough information to answer the question, you must explicitly say that in the final answer.\n\n"
        "=== CONTEXT ===\n"
        f"{context_text}\n\n"
        "=== QUESTION ===\n"
        f"{question}\n\n"
        "=== DRAFT ANSWER ===\n"
        f"{draft_answer}\n\n"
        "=== TASK ===\n"
        "1) Identify unsupported, speculative, or hallucinated parts of the draft answer.\n"
        "2) Rewrite the answer so that EVERY statement is supported by the context.\n"
        "3) The final answer MUST be clear, concise and in ENGLISH.\n"
        "4) After the main answer, add a short 'SOURCES:' section where you briefly list which parts of the context support the key points of the answer.\n"
        "   For example: 'SOURCES: paragraph about Pavlov's experiments; section describing behaviorism; part about Taylor's management theory'.\n"
        "5) Do NOT invent page numbers; use only descriptions that can be inferred from the context.\n\n"
        "=== OUTPUT FORMAT (STRICT) ===\n"
        "FINAL ANSWER:\n"
        "- <your final, verified answer here>\n\n"
        "SOURCES:\n"
        "- <short description of the main supporting fragment 1>\n"
        "- <short description of the main supporting fragment 2>\n"
        "- ...\n\n"
        "Return ONLY the 'FINAL ANSWER' and 'SOURCES' sections, without any extra commentary.\n"
    )
    return prompt


Запусти Block 13A.

Запусти Block 12A не нужно заново (если ядро не перезапускалось) — мы просто переопределили функцию, она уже в памяти.

Запусти Block 12B и 12C, если ты их менял после этого (не обязательно, но безопасно).

Запусти Block 8

я уже засыпаю мм

## лёгкий sentence-level trimming

In [ ]:
# === Block 14. Sentence-level trimming контекста под вопрос ===
# ИДЕЯ:
#   - каждый выбранный чанк мы режем на предложения;
#   - для каждого предложения считаем "похожесть" на вопрос по пересечению ключевых слов;
#   - оставляем N самых релевантных предложений, сохраняя исходный порядок;
#   - если текст и так короткий — оставляем как есть.
#
# ПЛЮС:
#   - меньше шума в контексте → LLM проще держаться фактов;
#   - self-RAG видит более сфокусированный текст.
#
# ВАЖНО:
#   - мы НЕ трогаем FAISS/BM25/reranker, только финальный context_text.

import re
from collections import Counter

# Небольшой набор английских стоп-слов, чтобы не учитывать "the, and, of, to, ..."
_TRIM_STOPWORDS = {
    # русские
    "и", "в", "во", "не", "что", "он", "она", "оно", "они",
    "как", "а", "но", "yet", "да", "или", "либо",
    "к", "ко", "от", "до", "из", "за", "над", "под", "при", "по", "о", "об", "обо",
    "на", "с", "со", "у", "про", "для", "без", "через", "между",
    "это", "этот", "эта", "это", "эти", "того", "той", "тех",
    "тот", "та", "те", "там", "тут", "здесь",
    "же", "уж", "ли", "бы", "то", "же", "уже", "ещё", "ещё", "тоже",
    "быть", "есть", "был", "была", "были", "будет", "будут",
    "мы", "вы", "ты", "они", "он", "она",
    "мой", "моя", "мои", "твой", "твоя", "твои", "наш", "наша", "наши",
    "их", "его", "ее", "её", "сам", "сама", "сами",
    "там", "здесь", "сюда", "туда",
    # английские (на всякий случай — вдруг смешанный текст)
    "the", "a", "an", "and", "or", "for", "of", "to", "in", "on", "at",
    "is", "are", "was", "were", "be", "been", "being",
    "this", "that", "these", "those",
    "by", "with", "from", "as", "it", "its",
    "about", "into", "over", "under", "between", "through",
    "he", "she", "they", "them", "his", "her", "their",
}

def _simple_tokenize(text: str):
    """
    Очень простой токенайзер:
    - приводим к нижнему регистру
    - выкидываем все, что не буква/цифра/пробел
    - сплитим по пробелам
    """
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    tokens = [t for t in text.split() if t]
    return tokens

def _keywords(text: str):
    """
    Ключевые слова: токены без стоп-слов и длиной >= 3.
    """
    toks = _simple_tokenize(text)
    return [t for t in toks if len(t) >= 3 and t not in _TRIM_STOPWORDS]

def split_into_sentences(text: str):
    """
    Очень простой сплиттер на предложения:
    режем по .!? с сохранением базовой структуры.
    """
    # Заменяем перевод строки на пробел — часто в pdf они стоят внутри предложений
    text = text.replace("\n", " ")
    # Грубый сплит по конечным знакам. Это не идеально, но для учебника ок.
    parts = re.split(r"(?<=[.!?])\s+", text)
    sentences = [s.strip() for s in parts if s.strip()]
    return sentences

def score_sentence_relevance(sentence: str, question: str) -> float:
    """
    Оценка релевантности предложения к вопросу.
    Очень простой скор:
      score = |пересечение ключевых слов| / (1 + |ключевые слова предложения|)
    """
    q_kw = set(_keywords(question))
    s_kw = _keywords(sentence)
    if not s_kw or not q_kw:
        return 0.0
    inter = q_kw.intersection(s_kw)
    score = len(inter) / (1.0 + len(s_kw))
    return score

def trim_chunk_for_question(
    chunk_text: str,
    question: str,
    max_sentences: int = 3,
    min_chars: int = 300,
):
    """
    Основная функция тримминга чанка:
    - если чанк короткий (<= min_chars) — возвращаем как есть;
    - иначе:
        * режем на предложения
        * если предложений мало (<= max_sentences) — тоже возвращаем как есть
        * считаем score_sentence_relevance для каждого предложения
        * выбираем top-k предложений по score
        * сортируем их в исходном порядке и склеиваем обратно
        * если по каким-то причинам всё пусто — fallback к исходному тексту
    """
    text = chunk_text.strip()
    if len(text) <= min_chars:
        return text

    sentences = split_into_sentences(text)
    if len(sentences) <= max_sentences:
        return text

    # считаем скор для каждой фразы
    scored = []
    for idx, sent in enumerate(sentences):
        sc = score_sentence_relevance(sent, question)
        scored.append((idx, sent, sc))

    # если все скоры нулевые — лучше ничего не трогать
    if all(sc == 0.0 for _, _, sc in scored):
        return text

    # сортируем по score (по убыванию), берём top-k,
    # затем сортируем по исходному индексу, чтобы сохранить порядок
    scored_sorted = sorted(scored, key=lambda x: x[2], reverse=True)
    top = scored_sorted[:max_sentences]
    top_sorted_by_idx = sorted(top, key=lambda x: x[0])

    trimmed_sentences = [s for _, s, _ in top_sorted_by_idx]
    trimmed_text = " ".join(trimmed_sentences).strip()

    # на всякий случай fallback
    if len(trimmed_text) < 1:
        return text

    return trimmed_text


# --- Переопределяем retrieve_relevant_chunks, чтобы использовать тримминг внутри ---
# ВАЖНО:
#   - предполагается, что уже определены:
#       * get_candidates_multiquery(...)
#       * rerank_chunks_with_ce(...)
#       * chunks_df
#       * MAX_CONTEXT_CHARS
#   - то есть этот блок надо запускать ПОСЛЕ Block 11A/11B/11C и 10A/10B.

ADAPTIVE_THRESHOLD = 0.0     # можно тюнить
MIN_CHUNKS = 3
MAX_CHUNKS = 8

def retrieve_relevant_chunks(
    query: str,
    top_k_faiss: int = 50,
    top_k_bm25: int = 50,
    alpha_faiss: float = 0.6,
    max_context_chars: int = MAX_CONTEXT_CHARS,
    trim_max_sentences: int = 3,
    trim_min_chars: int = 300,
):
    """
    Финальная версия:
    1) multi-query (paraphrases + step-back)
    2) hybrid search (FAISS + BM25)
    3) cross-encoder rerank
    4) Adaptive K (по score_ce)
    5) sentence-level trimming внутри каждого чанка
    6) склейка контекста
    """

    # 1. Multi-query candidates
    multi_candidates = get_candidates_multiquery(
        question=query,
        top_k_faiss=top_k_faiss,
        top_k_bm25=top_k_bm25,
        alpha_faiss=alpha_faiss,
    )

    if not multi_candidates:
        return {"chunks": [], "context_text": ""}

    # 2. Cross-encoder rerank
    reranked = rerank_chunks_with_ce(
        query=query,
        candidates=multi_candidates,
        top_k_final=MAX_CHUNKS * 3,   # сначала берём побольше, потом фильтруем
    )

    # 3. Adaptive K
    confident = [ch for ch in reranked if ch.get("score_ce", 0) >= ADAPTIVE_THRESHOLD]

    if len(confident) < MIN_CHUNKS:
        selected = reranked[:MIN_CHUNKS]
    else:
        selected = confident[:MAX_CHUNKS]

    # 4. Формируем context_text с триммингом предложений
    seen = set()
    parts = []
    final_chunks = []

    for ch in selected:
        cid = int(ch["chunk_id"])
        if cid in seen:
            continue
        seen.add(cid)

        row = chunks_df.loc[chunks_df["chunk_id"] == cid].iloc[0]
        raw_text = str(row["chunk_text"])

        trimmed = trim_chunk_for_question(
            chunk_text=raw_text,
            question=query,
            max_sentences=trim_max_sentences,
            min_chars=trim_min_chars,
        )

        final_chunks.append(ch)
        parts.append(trimmed)

    context_text = "\n\n".join(parts)
    if len(context_text) > max_context_chars:
        context_text = context_text[:max_context_chars]

    return {
        "chunks": final_chunks,
        "context_text": context_text,
    }

print("Sentence-level trimming для retrieve_relevant_chunks() активирован.")

